### 2. <u>Parallelization</u>:

In LangGraph, nodes typically execute in a sequence defined by edges, but when tasks don't depend on each othe's outputs, You can run them in parallel. This is achieved by:

* Defining multiple nodes that can operate independently.

* Connecting them to a common starting point (example: START or another node).

* Merging their outputs into a downstream node if needed.

In [1]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model='gemma3:270m')
response = llm.invoke("Hi")
response

AIMessage(content='Hi there! How can I help you today?\n', additional_kwargs={}, response_metadata={'model': 'gemma3:270m', 'created_at': '2025-09-30T05:49:30.7796522Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1903274800, 'load_duration': 1499149200, 'prompt_eval_count': 10, 'prompt_eval_duration': 113968900, 'eval_count': 12, 'eval_duration': 254427900, 'model_name': 'gemma3:270m'}, id='run--cd356bb7-de2e-4657-9fda-66b61b2ceefe-0', usage_metadata={'input_tokens': 10, 'output_tokens': 12, 'total_tokens': 22})

In [6]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END

class State(TypedDict):
    topic:str
    char:str
    settings:str
    premise:str
    story_intro:str

def generate_char(state:State):
    """Generates character description"""
    msg = llm.invoke(f"Create 2 character names and brief their traits for a story about {state['topic']}")
    return {'char':msg.content}

def generate_settings(state:State):
    """Generates story settings"""
    msg = llm.invoke(f"Describe a vivid setting for a story about {state['topic']}")
    return {'settings':msg.content}

def generate_premises(state:State):
    """Generates story premise"""
    msg = llm.invoke(f"Write an engaging, emotional 2 sentence plot premise for a story about {state['topic']}")
    return {'premise':msg.content}

def generate_intro(state:State):
    """Generates story intro"""
    msg = llm.invoke(f"""From the details given below, generate an emotional, engaging story on the topic {state['topic']}:
                     character_description: {state['char']}
                    story_setting:{state['settings']}
                    story_premise:{state['premise']}
                     """)
    return {'story_intro':msg.content}

In [9]:
builder = StateGraph(State)

builder.add_node("char",generate_char)
builder.add_node("setting",generate_settings)
builder.add_node("premise",generate_premises)
builder.add_node("intro",generate_intro)

builder.add_edge(START,"char")
builder.add_edge(START,"setting")
builder.add_edge(START,"premise")
builder.add_edge("char","intro")
builder.add_edge("setting","intro")
builder.add_edge("premise","intro")
builder.add_edge("intro",end_key=END)

builder_graph = builder.compile()

In [18]:
response = builder_graph.invoke({'topic':"Space survivor"})

In [19]:
from IPython.display import Markdown

Markdown(response['story_intro'])

The airlock hissed, a mournful sigh that echoed through the cramped confines of the 'Stardust Drifter'. Kepler, her face etched with a mixture of weariness and a flicker of hope, gripped the rusty control panel. The familiar hum of the engine, usually a comforting symphony, now felt like a discordant note, a constant reminder of the world she’d left behind. This wasn’t just a job; it was a calling, a desperate attempt to rebuild something from the ashes of what was.

Her fingers, stained with the grime of countless journeys, traced the intricate patterns of the constellations. The stars, once symbols of celestial beauty, now seemed to mock her with their malevolent glow. She’d been born into a world of endless possibilities, a world where the stars held the key to understanding the universe. But the path had been paved with darkness, with whispers of despair that clung to her like a shroud.

The 'Stardust Drifter', a battered but trusty vessel, was her refuge, her shield against the encroaching darkness. The silence, once a source of comfort, now felt like a suffocating blanket. She had learned to navigate the treacherous currents of the cosmos, to chart the course of the stars, to find her own rhythm in the vast, indifferent universe.

The initial days were a blur of exploration. She’d learned to navigate the treacherous canyons, to decipher the cryptic language of the stars, to unravel the secrets hidden within the shimmering dust. She’d discovered a knack for observation, a unique ability to perceive the subtle nuances of light and shadow. Her eyes, usually hidden by the visor, were now filled with a depth of understanding that surpassed her years of solitude.

But the darkness was relentless. The loneliness, the fear, the gnawing emptiness - they all clung to her like a second skin. She found herself repeating the same phrases, repeating the same patterns, repeating the same songs, trying to find a sense of belonging in the face of overwhelming despair.

One evening, huddled in the dusty tunnel, she stumbled upon a hidden chamber. A single, flickering torch cast a weak, ethereal light on a crystalline structure that pulsed with a faint, rhythmic energy. It was a place of immense power, a place where the universe was not just a collection of stars, but a living, breathing entity.

Hesitantly, she approached the structure. The air around it crackled with energy, and a sense of profound peace washed over her. She felt a connection to the universe, a sense of being part of something larger than herself.

That night, she slept soundly, her mind free from the constant barrage of cosmic noise. She began to explore the chamber, drawn to the crystalline structure and the pulsating energy. She started to learn about the constellations, to decipher the language of the stars, to understand the mysteries of the universe.

Slowly, she began to feel a sense of purpose. She was no longer just a survivor, a solitary figure trapped in a world of endless possibilities. She was a guardian of the cosmos, a protector of the fragile balance between light and darkness.

The journey was far from over. The darkness would continue to consume her, but she was no longer afraid. She had found her strength, her purpose, and she was ready to face whatever the universe threw her way. The silence was still there, but now it was a quiet, comforting symphony, a reminder of the incredible journey she had undertaken. She was a survivor, and she would continue to navigate the cosmic dust, to learn from the stars, and to protect the universe from the encroaching darkness.